In [5]:
from google.colab import drive
drive.mount('/content/drive')


!pip install pdfplumber

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 60.3 MB/s eta 0:00:00


In [100]:
import pdfplumber
import csv
import string
import unicodedata
import pandas as pd
from collections import defaultdict



path = '/content/drive/MyDrive/Colab Notebooks/NLP/Los_Miserables/'
pdf_name = 'Los-miserables.pdf'
csv_name = 'Los-miserables.csv'
csv_path = path + csv_name
pd.set_option('display.max_colwidth', None)

In [97]:
with pdfplumber.open(path + pdf_name) as pdf:
  with open(path + csv_name, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    for page in pdf.pages:
      text = page.extract_text()
      if text:
        writer.writerow([text])

In [102]:
df = pd.read_csv(csv_path, header=None)
df.columns = ['page']
df.head(5)


,page
0,"Los miserables I\nHugo, Victor\nNovela\nSe reconocen los derechos morales de Hugo, Victor.\nObra de dominio público.\nDistribución gratuita. Prohibida su venta y distribución en medios ajenos a la Fundación Carlos Slim.\nFundación Carlos Slim\nLago Zúrich. Plaza Carso II. Piso 5. Col. Ampliación Granada\nC. P. 11529, Ciudad de México. México.\ncontacto@pruebat.org\n2"
1,PRIMERA PARTE\nFantine\n3
2,LIBRO PRIMERO\nUn justo\n4
3,"I\nEl señor Myriel\nEn 1815, monseñor Charles-François-Bienvenu Myriel era obispo de Digne. Era un\nanciano de cerca de setenta y cinco años y ocupaba la sede de Digne desde 1806.\nAunque este detalle no interesa en manera alguna al fondo de lo que vamos a\nreferir, quizá no será inútil, aunque no sea más que para ser exactos en todo, indicar\naquí los rumores y las habladurías que habían circulado acerca de su persona, en el\nmomento en que llegó a la diócesis. Verdadero o falso, lo que de los hombres se dice\nocupa en su vida, y sobre todo en su destino, tanto lugar como lo que hacen.\nMonseñor Myriel era hijo de un consejero del departamento de Aix; nobleza de toga.\nDecíase de él que su padre, reservándole para heredar su puesto, le había casado muy\npronto, a los dieciocho o veinte años, siguiendo una costumbre muy extendida entre\nlas familias parlamentarias. Charles Myriel, no obstante este matrimonio, había dado —\ndecíase— mucho que hablar. Era de buena presencia, aunque de estatura pequeña,\nelegante, gracioso, inteligente; toda la primera parte de su vida había estado\nconsagrada al mundo y a las galanterías.\nSobrevino la Revolución, precipitáronse los sucesos; las familias parlamentarias,\ndiezmadas, perseguidas, acosadas, se dispersaron, y el señor Charles Myriel, en los\nprimeros días de la Revolución, emigró a Italia. Su mujer murió allí de una enfermedad\ndel pecho, que padecía desde mucho tiempo atrás. No tenían hijos. ¿Qué pasó,\ndespués, en la vida del señor Myriel? El hundimiento de la antigua sociedad francesa,\nla caída de su propia familia, los trágicos espectáculos del 93, más espantosos aún\nquizá para los emigrados, que los veían de lejos con el aumento que les prestaba el\nterror, ¿hicieron germinar tal vez en su alma ideas de renuncia y de soledad? En medio\nde las distracciones y de los afectos que ocupaban su vida, ¿fue súbitamente herido\npor uno de estos golpes misteriosos y terribles que algunas veces llegan a derribar,\nlacerándole el corazón, al hombre a quien las catástrofes públicas no conmoverían, si\nle hiriesen en su existencia o en su hacienda? Nadie hubiera podido decirlo; sólo se\nsabía que, a su vuelta de Italia, era sacerdote.\n5"
4,"En 1804, el señor Myriel era cura párroco de B. (Brignolles). Era ya anciano y vivía\nen un absoluto retiro.\nHacia la época de la Coronación, un pequeño asunto de su parroquia, no se sabe a\npunto fijo cuál, le llevó a París. Entre otras personas poderosas, fue a solicitar amparo\npara sus feligreses al cardenal Fesch. Un día en que el emperador fue a visitar a su tío,\nel digno párroco, que esperaba en la antecámara, se halló ante Su Majestad.\nNapoleón, al ver que aquel anciano le observaba con cierta curiosidad, se volvió y\npreguntó bruscamente:\n—¿Quién es este buen hombre que me mira?\n—Señor —dijo el señor Myriel—, vos miráis a un buen hombre, y yo miro a un gran\nhombre. Ambos podemos sacar provecho.\nAquella misma noche, el emperador preguntó al cardenal el nombre de aquel cura\ny, algún tiempo después, el señor Myriel quedó sorprendido al enterarse de que había\nsido nombrado obispo de Digne.\n¿Qué había de verdad, por lo demás, en las habladurías sobre la primera parte de\nla vida de monseñor Myriel? Nadie lo sabía. Pocas familias habían conocido a la de\nMyriel antes de la Revolución.\nMonseñor Myriel debía sufrir la suerte de todos los recién llegados a una pequeña\nciudad, donde hay muchas bocas que hablan, y muy pocas cabezas que piensan.\nDebía sufrirla, aunque fuera obispo, y precisamente porque era obi

In [104]:
df['page'] = df['page'].str.replace('\n', ' ', regex=False).replace('\r', ' ', regex=False)
# Eliminar signos de puntuación
df['page'] = df['page'].str.translate(str.maketrans('', '', string.punctuation))
# Normalizar acentos a caracteres ASCII sin acentos
df['page'] = df['page'].str.normalize('NFKD').str.encode('ASCII', 'ignore').str.decode('ASCII')
# Eliminar espacios extra (múltiples espacios se convierten en uno solo)
df['page'] = df['page'].str.replace(r'\s+', ' ', regex=True)
# Convertir todo el texto a minúsculas
df['page'] = df['page'].str.lower()

df['page'].head(5)

,page
0,los miserables i hugo victor novela se reconocen los derechos morales de hugo victor obra de dominio publico distribucion gratuita prohibida su venta y distribucion en medios ajenos a la fundacion carlos slim fundacion carlos slim lago zurich plaza carso ii piso 5 col ampliacion granada c p 11529 ciudad de mexico mexico contactopruebatorg 2
1,primera parte fantine 3
2,libro primero un justo 4
3,i el senor myriel en 1815 monsenor charlesfrancoisbienvenu myriel era obispo de digne era un anciano de cerca de setenta y cinco anos y ocupaba la sede de digne desde 1806 aunque este detalle no interesa en manera alguna al fondo de lo que vamos a referir quiza no sera inutil aunque no sea mas que para ser exactos en todo indicar aqui los rumores y las habladurias que habian circulado acerca de su persona en el momento en que llego a la diocesis verdadero o falso lo que de los hombres se dice ocupa en su vida y sobre todo en su destino tanto lugar como lo que hacen monsenor myriel era hijo de un consejero del departamento de aix nobleza de toga deciase de el que su padre reservandole para heredar su puesto le habia casado muy pronto a los dieciocho o veinte anos siguiendo una costumbre muy extendida entre las familias parlamentarias charles myriel no obstante este matrimonio habia dado deciase mucho que hablar era de buena presencia aunque de estatura pequena elegante gracioso inteligente toda la primera parte de su vida habia estado consagrada al mundo y a las galanterias sobrevino la revolucion precipitaronse los sucesos las familias parlamentarias diezmadas perseguidas acosadas se dispersaron y el senor charles myriel en los primeros dias de la revolucion emigro a italia su mujer murio alli de una enfermedad del pecho que padecia desde mucho tiempo atras no tenian hijos que paso despues en la vida del senor myriel el hundimiento de la antigua sociedad francesa la caida de su propia familia los tragicos espectaculos del 93 mas espantosos aun quiza para los emigrados que los veian de lejos con el aumento que les prestaba el terror hicieron germinar tal vez en su alma ideas de renuncia y de soledad en medio de las distracciones y de los afectos que ocupaban su vida fue subitamente herido por uno de estos golpes misteriosos y terribles que algunas veces llegan a derribar lacerandole el corazon al hombre a quien las catastrofes publicas no conmoverian si le hiriesen en su existencia o en su hacienda nadie hubiera podido decirlo solo se sabia que a su vuelta de italia era sacerdote 5
4,en 1804 el senor myriel era cura parroco de b brignolles era ya anciano y vivia en un absoluto retiro hacia la epoca de la coronacion un pequeno asunto de su parroquia no se sabe a punto fijo cual le llevo a paris entre otras personas poderosas fue a solicitar amparo para sus feligreses al cardenal fesch un dia en que el emperador fue a visitar a su tio el digno parroco que esperaba en la antecamara se hallo ante su majestad napoleon al ver que aquel anciano le observaba con cierta curiosidad se volvio y pregunto bruscamente quien es este buen hombre que me mira senor dijo el senor myriel vos mirais a un buen hombre y yo miro a un gran hombre ambos podemos sacar provecho aquella misma noche el emperador pregunto al cardenal el nombre de aquel cura y algun tiempo despues el senor myriel quedo sorprendido al enterarse de que habia sido nombrado obispo de digne que habia de verdad por lo demas en las habladurias sobre la primera parte de la vida de monsenor myriel nadie lo sabia pocas familias habian conocido a la de myriel antes de la revolucion monsenor myriel debia sufrir la suerte de todos los recien llegados a una pequena ciudad donde hay muchas bocas que hablan y muy pocas cabezas que piensan debia sufrirla aunque fuera obispo y precisamente porque era obispo pero despues de todo las habladurias en las que se mezclaba su nombre no eran mas que habladurias ruido frases palabras menos aun que palabras palabrerias como se dice en el energico lenguaje

In [107]:
vocabulario = {}
for page in df['page']:
  for word in page.split(' '):
    try:
      vocabulario[word] += 1
    except:
      vocabulario[word] = 1

In [110]:
vocabulario = dict(sorted(vocabulario.items(), key=lambda item: item[1], reverse=True))
vocabulario_df = pd.DataFrame(list(vocabulario.items()), columns=['Word', 'Frequency'])

vocabulario_df.to_parquet(path + 'vocabulario.parquet', engine='pyarrow')

In [111]:
vocabulario_df.head()

,Word,Frequency
0,de,5325
1,la,3918
2,que,3818
3,el,3394
4,y,3123


In [114]:
pd.set_option('display.max_rows', None)
total_palabras = vocabulario_df['Frequency'].sum()
print(f'Total words in the original text: {total_palabras}')
print('*********')
num_palabras_diferentes = vocabulario_df.shape[0]
print(f'Number of different words in the vocabulary: {num_palabras_diferentes}')
print('*********')
top_100_frequent_words = vocabulario_df.sort_values(by='Frequency', ascending=False).head(100)
print('Top 100 most frequent words:')
print(top_100_frequent_words)
print('*********')
bottom_100_frequent_words = vocabulario_df.sort_values(by='Frequency', ascending=True).head(100)
print('Top 100 least frequent words:')
print(bottom_100_frequent_words)

Total words in the original text: 109703
*********
Number of different words in the vocabulary: 13453
*********
Top 100 most frequent words:
         Word  Frequency
0          de       5325
1          la       3918
2         que       3818
3          el       3394
4           y       3123
5          en       2836
6           a       2489
7          se       1681
8          un       1601
9          no       1499
10        los       1353
11        una       1319
12         su       1245
13        por        936
14        las        935
15        con        924
16      habia        858
17        del        813
18         al        756
19         es        749
20         lo        719
21         le        667
22        era        650
23       como        572
24        mas        513
25       para        504
26      senor        447
27       esta        414
28       pero        372
29     hombre        363
30         si        358
31        sus        344
32       todo        327
33       